In [1]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMClassifier
import os
import seaborn as sns
from wordcloud import WordCloud
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("C:/curos/ia/deepLearning/deepLearning/machine/URLDectetor/NOSE/combined_uniqueMAlw.csv")




In [3]:
# df["url"][0]

In [4]:
df.type.value_counts()

0    395685
1    183401
Name: type, dtype: int64

In [5]:
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))

In [6]:
df

,url,type,use_of_ip
0,http://123.10.136.52:44381/Mozi.m,1,0
1,http://117.220.150.175:43986/bin.sh,1,0
2,http://115.56.173.64:60109/i,1,0
3,http://117.213.84.219:56997/i,1,0
4,http://223.13.85.46:34095/bin.sh,1,0
...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0
579084,http://atualizapj.com/,1,0


In [7]:
from urllib.parse import urlparse
import re

def abnormal_url(url):
    try:
        hostname = urlparse(url).hostname
        # Escapamos el hostname para usarlo en una expresión regular
        pattern = re.escape(str(hostname))
        match = re.search(pattern, url)
        if match:
            # Si encuentra el hostname en la URL, no es anormal
            return 0
        else:
            # Si no encuentra el hostname, es anormal
            return 1
    except Exception as e:
        # Si ocurre un error al parsear la URL, considerarla anormal
        return 1

# Aplicamos la función al dataframe
df['abnormal_url'] = df['url'].apply(abnormal_url)


In [8]:
df

,url,type,use_of_ip,abnormal_url
0,http://123.10.136.52:44381/Mozi.m,1,0,0
1,http://117.220.150.175:43986/bin.sh,1,0,0
2,http://115.56.173.64:60109/i,1,0,0
3,http://117.213.84.219:56997/i,1,0,0
4,http://223.13.85.46:34095/bin.sh,1,0,0
...,...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0,0
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0
579084,http://atualizapj.com/,1,0,0


In [9]:
from googlesearch import search

In [10]:
def google_index(url):
    site = search(url, 5)
    return 1 if site else 0

df["google_index"]= df["url"].apply(lambda i: google_index(i))

In [11]:
def count_dot(url):
    count_dot = url.count(".")
    return count_dot

df["count."]= df["url"].apply(lambda i: count_dot(i))
df.head()

,url,type,use_of_ip,abnormal_url,google_index,count.
0,http://123.10.136.52:44381/Mozi.m,1,0,0,1,4
1,http://117.220.150.175:43986/bin.sh,1,0,0,1,4
2,http://115.56.173.64:60109/i,1,0,0,1,3
3,http://117.213.84.219:56997/i,1,0,0,1,3
4,http://223.13.85.46:34095/bin.sh,1,0,0,1,4


In [12]:
def count_www(url):
    url.count('www')
    return url.count('www')

df['count-www'] = df['url'].apply(lambda i: count_www(i))

def count_atrate(url):
     
    return url.count('@')

df['count@'] = df['url'].apply(lambda i: count_atrate(i))


def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))

def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

df['count_embed_domian'] = df['url'].apply(lambda i: no_of_embed(i))


def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
    
    
df['short_url'] = df['url'].apply(lambda i: shortening_service(i))




In [13]:
df

,url,type,use_of_ip,abnormal_url,google_index,count.,count-www,count@,count_dir,count_embed_domian,short_url
0,http://123.10.136.52:44381/Mozi.m,1,0,0,1,4,0,0,1,0,0
1,http://117.220.150.175:43986/bin.sh,1,0,0,1,4,0,0,1,0,0
2,http://115.56.173.64:60109/i,1,0,0,1,3,0,0,1,0,0
3,http://117.213.84.219:56997/i,1,0,0,1,3,0,0,1,0,0
4,http://223.13.85.46:34095/bin.sh,1,0,0,1,4,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0,0,1,2,0,0,3,0,1
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,2,0,0,12,0,0
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,1,0,0,12,0,0
579084,http://atualizapj.com/,1,0,0,1,1,0,0,1,0,0


In [14]:
def count_https(url):
    return url.count('https')

df['count-https'] = df['url'].apply(lambda i : count_https(i))

def count_http(url):
    return url.count('http')

df['count-http'] = df['url'].apply(lambda i : count_http(i))


In [15]:
df

,url,type,use_of_ip,abnormal_url,google_index,count.,count-www,count@,count_dir,count_embed_domian,short_url,count-https,count-http
0,http://123.10.136.52:44381/Mozi.m,1,0,0,1,4,0,0,1,0,0,0,1
1,http://117.220.150.175:43986/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1
2,http://115.56.173.64:60109/i,1,0,0,1,3,0,0,1,0,0,0,1
3,http://117.213.84.219:56997/i,1,0,0,1,3,0,0,1,0,0,0,1
4,http://223.13.85.46:34095/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0,0,1,2,0,0,3,0,1,0,1
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,2,0,0,12,0,0,0,1
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,1,0,0,12,0,0,0,1
579084,http://atualizapj.com/,1,0,0,1,1,0,0,1,0,0,0,1


In [16]:
def count_per(url):
    return url.count('%')

df['count%'] = df['url'].apply(lambda i : count_per(i))

def count_ques(url):
    return url.count('?')

df['count?'] = df['url'].apply(lambda i: count_ques(i))

def count_hyphen(url):
    return url.count('-')

df['count-'] = df['url'].apply(lambda i: count_hyphen(i))

def count_equal(url):
    return url.count('=')

df['count='] = df['url'].apply(lambda i: count_equal(i))

def url_length(url):
    return len(str(url))


#Length of URL
df['url_length'] = df['url'].apply(lambda i: url_length(i))

#Hostname Length
def hostname_length(url):
    return len(urlparse(url).netloc)

df['hostname_length'] = df['url'].apply(lambda i: hostname_length(i))


In [17]:
df

,url,type,use_of_ip,abnormal_url,google_index,count.,count-www,count@,count_dir,count_embed_domian,short_url,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length
0,http://123.10.136.52:44381/Mozi.m,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,33,19
1,http://117.220.150.175:43986/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,35,21
2,http://115.56.173.64:60109/i,1,0,0,1,3,0,0,1,0,0,0,1,0,0,0,0,28,19
3,http://117.213.84.219:56997/i,1,0,0,1,3,0,0,1,0,0,0,1,0,0,0,0,29,20
4,http://223.13.85.46:34095/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,32,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0,0,1,2,0,0,3,0,1,0,1,0,0,1,0,43,11
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,2,0,0,12,0,0,0,1,0,0,0,1,159,13
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,1,0,0,12,0,0,0,1,0,0,0,1,147,13
579084,http://atualizapj.com/,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,22,14


In [18]:
def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0
df['sus_url'] = df['url'].apply(lambda i: suspicious_words(i))


def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits


df['count-digits']= df['url'].apply(lambda i: digit_count(i))


def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters


df['count-letters']= df['url'].apply(lambda i: letter_count(i))

In [19]:
df

,url,type,use_of_ip,abnormal_url,google_index,count.,count-www,count@,count_dir,count_embed_domian,short_url,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length,sus_url,count-digits,count-letters
0,http://123.10.136.52:44381/Mozi.m,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,33,19,0,15,9
1,http://117.220.150.175:43986/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,35,21,0,17,9
2,http://115.56.173.64:60109/i,1,0,0,1,3,0,0,1,0,0,0,1,0,0,0,0,28,19,0,15,5
3,http://117.213.84.219:56997/i,1,0,0,1,3,0,0,1,0,0,0,1,0,0,0,0,29,20,0,16,5
4,http://223.13.85.46:34095/bin.sh,1,0,0,1,4,0,0,1,0,0,0,1,0,0,0,0,32,18,0,14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579081,http://ecct-it.com/docmmmnn/aptgd/index.php,1,0,0,1,2,0,0,3,0,1,0,1,0,0,1,0,43,11,0,0,34
579082,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,2,0,0,12,0,0,0,1,0,0,0,1,159,13,0,21,118
579083,http://faboleena.com/js/infortis/jquery/plugin...,1,0,0,1,1,0,0,12,0,0,0,1,0,0,0,1,147,13,0,20,109
579084,http://atualizapj.com/,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,22,14,0,0,17


In [20]:
from urllib.parse import urlparse
from tld import get_tld
import os.path

In [21]:
#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

df['fd_length'] = df['url'].apply(lambda i: fd_length(i))

#Length of Top Level Domain
df['tld'] = df['url'].apply(lambda i: get_tld(i,fail_silently=True))


def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))

In [22]:
df = df.drop("tld",1)

C:\Users\fanta\AppData\Local\Temp\ipykernel_2112\2551734815.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop("tld",1)


In [23]:
df.columns

Index(['url', 'type', 'use_of_ip', 'abnormal_url', 'google_index', 'count.',
       'count-www', 'count@', 'count_dir', 'count_embed_domian', 'short_url',
       'count-https', 'count-http', 'count%', 'count?', 'count-', 'count=',
       'url_length', 'hostname_length', 'sus_url', 'count-digits',
       'count-letters', 'fd_length', 'tld_length'],
      dtype='object')

In [24]:
df["type"].value_counts()

0    395685
1    183401
Name: type, dtype: int64

In [25]:
#Target Encoding
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["type_code"] = lb_make.fit_transform(df["type"])
df["type_code"].value_counts()

0    395685
1    183401
Name: type_code, dtype: int64

In [26]:
#Creation of Feature & Target
#Predictor Variables
# filtering out google_index as it has only 1 value
X = df[['use_of_ip','abnormal_url', 'count.', 'count-www', 'count@',
       'count_dir', 'count_embed_domian', 'short_url', 'count-https',
       'count-http', 'count%', 'count?', 'count-', 'count=', 'url_length',
       'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count-digits',
       'count-letters']]

#Target Variable
y = df['type_code']

In [27]:
X

,use_of_ip,abnormal_url,count.,count-www,count@,count_dir,count_embed_domian,short_url,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length,sus_url,fd_length,tld_length,count-digits,count-letters
0,0,0,4,0,0,1,0,0,0,1,0,0,0,0,33,19,0,6,-1,15,9
1,0,0,4,0,0,1,0,0,0,1,0,0,0,0,35,21,0,6,-1,17,9
2,0,0,3,0,0,1,0,0,0,1,0,0,0,0,28,19,0,1,-1,15,5
3,0,0,3,0,0,1,0,0,0,1,0,0,0,0,29,20,0,1,-1,16,5
4,0,0,4,0,0,1,0,0,0,1,0,0,0,0,32,18,0,6,-1,14,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579081,0,0,2,0,0,3,0,1,0,1,0,0,1,0,43,11,0,8,3,0,34
579082,0,0,2,0,0,12,0,0,0,1,0,0,0,1,159,13,0,2,3,21,118
579083,0,0,1,0,0,12,0,0,0,1,0,0,0,1,147,13,0,2,3,20,109
579084,0,0,1,0,0,1,0,0,0,1,0,0,0,0,22,14,0,0,3,0,17


In [28]:
X.columns

Index(['use_of_ip', 'abnormal_url', 'count.', 'count-www', 'count@',
       'count_dir', 'count_embed_domian', 'short_url', 'count-https',
       'count-http', 'count%', 'count?', 'count-', 'count=', 'url_length',
       'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count-digits',
       'count-letters'],
      dtype='object')

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2,shuffle=True, random_state=5)

In [30]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy="median")
# #ahora llenamos los valores nulos
# x_train_prep = imputer.fit_transform(X_train)
# x_test_prep = imputer.fit_transform(X_test)

In [31]:
# x_train_prep = pd.DataFrame(x_train_prep, columns= X_train.columns, index=y_train.index)
# x_test_prep = pd.DataFrame(x_test_prep, columns=  X_test.columns, index= y_test.index)

In [32]:
# print(x_test_prep.head(10))

In [33]:
#print(x_test_prep.info())

In [34]:
is_null = X_train.isna().any()
print(is_null[is_null])

Series([], dtype: bool)


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier(n_estimators=100,max_features='sqrt')
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test,y_pred_rf,target_names=['benign','malicious']))


score =accuracy_score(y_test, y_pred_rf)
print("accuracy:   %0.3f" % score)

              precision    recall  f1-score   support

      benign       1.00      0.99      0.99     79138
   malicious       0.99      0.99      0.99     36680

    accuracy                           0.99    115818
   macro avg       0.99      0.99      0.99    115818
weighted avg       0.99      0.99      0.99    115818

accuracy:   0.992


In [36]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score

# Ajustamos el clasificador para una clasificación binaria
lgb = LGBMClassifier(
    objective='binary',
    boosting_type='gbdt',
    n_jobs=5,
    silent=True,
    random_state=5,
    learning_rate=0.1,
    n_estimators=100,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    num_leaves=31,
    lambda_l1=0,
    lambda_l2=0
)

# Entrenamos el modelo
LGB_C = lgb.fit(X_train, y_train)

# Hacemos predicciones
y_pred_lgb = LGB_C.predict(X_test)

# Imprimimos el reporte de clasificación
print(classification_report(y_test, y_pred_lgb, target_names=['benign', 'malicious']))

# Calculamos la precisión
score = accuracy_score(y_test, y_pred_lgb)
print("Accuracy: %0.3f" % score)

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 146721, number of negative: 316547
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012343 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory

In [37]:
# xgb_c = xgb.XGBClassifier(n_estimators= 100)
# xgb_c.fit(X_train,y_train)
# y_pred_x = xgb_c.predict(X_test)
# print(classification_report(y_test,y_pred_x,target_names=['benign', 'defacement','phishing','malware']))


# score =accuracy_score(y_test, y_pred_x)
# print("accuracy:   %0.3f" % score)

In [38]:
df.columns

Index(['url', 'type', 'use_of_ip', 'abnormal_url', 'google_index', 'count.',
       'count-www', 'count@', 'count_dir', 'count_embed_domian', 'short_url',
       'count-https', 'count-http', 'count%', 'count?', 'count-', 'count=',
       'url_length', 'hostname_length', 'sus_url', 'count-digits',
       'count-letters', 'fd_length', 'tld_length', 'type_code'],
      dtype='object')

In [39]:
from joblib import dump

# model es tu modelo entrenado
dump(lgb, 'model_filenameG.joblib')

['model_filenameG.joblib']

In [40]:
#save model
from joblib import load

model = load('model_filenameG.joblib')


In [41]:
def main(url):
    
    status = []
    
    status.append(having_ip_address(url))
    status.append(abnormal_url(url))
    status.append(count_dot(url))
    status.append(count_www(url))
    status.append(count_atrate(url))
    status.append(no_of_dir(url))
    status.append(no_of_embed(url))
    
    status.append(shortening_service(url))
    status.append(count_https(url))
    status.append(count_http(url))
    
    status.append(count_per(url))
    status.append(count_ques(url))
    status.append(count_hyphen(url))
    status.append(count_equal(url))
    
    status.append(url_length(url))
    status.append(hostname_length(url))
    status.append(suspicious_words(url))
    status.append(digit_count(url))
    status.append(letter_count(url))
    status.append(fd_length(url))
    tld = get_tld(url,fail_silently=True)
      
    status.append(tld_length(tld))
    
    
    

    return status

In [42]:
def get_prediction_from_url(test_url):
    features_test = main(test_url)
    # Due to updates to scikit-learn, we now need a 2D array as a parameter to the predict function.
    features_test = np.array(features_test).reshape((1, -1))

    

    pred = model.predict(features_test)
    print(int(pred[0]))
    if int(pred[0]) == 0:
        
        res="SAFE"
        return res
    else:
        res="PHISHING"
        return res


    
    

In [43]:
urls = ['https://chat.openai.com/c/8cf3146f-73a3-4c25-b39c-95abd6afdca4','https://wisdomml.in/malicious-url-detection-using-machine-learning-in-python/']
for url in urls:
     print(get_prediction_from_url(url))

[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
1
PHISHING
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0
SAFE
